In [6]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import math
import csv
from csv import writer
import concurrent.futures
import random

In [4]:
#scrape from freeproxy.world
url = "https://www.freeproxy.world/"
proxylist =[]
#filter through all pages
for i in range(1,75):
    print(i)
    try:
        #filter to http proxy
        querystring = {"type":"http","anonymity":"","country":"","speed":"","port":"","page":f"{i}"}

        payload = ""
        headers = {"User-Agent": "insomnia/8.4.5"}

        #site request
        site = requests.request("GET", url, data=payload, headers=headers, params=querystring)
        soup = BeautifulSoup(site.content, 'html.parser')
        trs = soup.find_all('tr')
        for tr in trs:
            try:
                ip = tr.find('td',class_=re.compile('show-ip-div')).text.strip()
                port = tr.find('a').text.strip()
                proxylist.append(f'{ip}:{port}')
            except:
                pass
    except:
        pass
print(len(proxylist))




#scrape free proxies from proxyscrape.com
headers = {"User-Agent": "insomnia/8.4.5"}
#site request
url = 'https://proxyscrape.com/free-proxy-list'
site = requests.get(url,headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
#scrape http proxy download link
divs = soup.find_all('div', class_=re.compile('itemcard downloadcard'))
for div in divs:
    h2 = div.find('h2')
    try:
        if(h2.text.strip()=='HTTP Proxies'):
            a = div.find('a')
            download = a['download_url']
    except:
        pass

#get content of download link
try:
    site = requests.get(download,headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    #add proxies to testlist
    tempproxylist = soup.text.strip().split('\r')
    for proxy in tempproxylist:
        proxylist.append(proxy)

    print(len(proxylist))
except:
    pass    
#scrape proxies from free-proxy-list.net
url = 'https://free-proxy-list.net/'
#site + soup
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
#scrape table
try:
    table = soup.find('div', class_=re.compile('table-responsive'))
    trlabels = table.find_all('tr')
except:
    pass
#scrape proxies and add to test list
try:
    for i in trlabels:
            tdlabels = i.find_all('td')
            if(tdlabels[6].text.strip()=='yes'):
                proxylist.append(tdlabels[0].text.strip())
except:
    pass
print(len(proxylist))
#scrape proxies from hidemy.io
url = 'https://hidemy.io/en/proxy-list/'
#site request + soup
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
table = soup.find('div',class_=re.compile('table_block'))
#scrape table
trlabels = table.find_all('tr')
#scrape proxies and add to test list
for i in trlabels:
    tdlabels = i.find_all('td')
    try:

        if(tdlabels[4].text.strip()=='HTTP'):
            proxylist.append(tdlabels[0].text.strip())
    except:
        pass
print(len(proxylist))





#test proxies
workingProxies = []
def extract(proxy):
    works = False
    try:
        r = requests.get('https://www.whatismybrowser.com/detect/what-is-my-ip-address', proxies={'http':proxy, 'https':proxy}, timeout=2)
        soup = BeautifulSoup(r.content, 'html.parser')
        div = soup.find('div', class_='detected_result')
        ip = div.find('div').text.strip()
        print(f'{r}, {ip}')
        works = True
        workingProxies.append(proxy)
        with open('working_proxies.csv', 'w', encoding='UTF8', newline='') as workingProx:
            writer = csv.writer(workingProx)
            for k in workingProxies:
                writer.writerow([k])

    except:
        print(proxy)
    return works

#excecute test faster
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(extract, list(set(proxylist)))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
3700
3700
3761
172.67.70.163:80172.67.176.250:80

172.67.161.219:80
141.101.122.40:80
172.67.206.142:80
185.162.228.178:80
141.101.121.133:80
172.67.229.19:80
45.131.4.184:80
45.12.31.16:80
172.67.180.205:80
164.38.155.19:80
45.12.30.247:80
172.67.3.69:80
141.193.213.196:80
185.162.228.88:80
172.67.70.153:80
172.67.128.39:80
172.64.86.93:80
66.235.200.147:80
104.20.233.70:80
172.67.181.155:80
188.166.197.129:3128
172.67.181.158:80
185.162.231.130:80
45.131.7.45:80
23.227.39.35:80
47.243.114.192:8180
172.67.3.62:80
157.254.53.50:80
172.64.149.43:80
172.67.93.9:80
23.227.38.100:80
188.114.96.104:80
172.67.182.136:80
31.43.179.158:80
172.67.167.14:80
173.245.49.41:80
63.141.128.42:80
69.84.182.25
51.210.54.186
45.12.31.202:80
138.68.60.8:8080
45.131.7.232:80
45.12.31.222:80
172.6

In [5]:
proxylist =[]
with open('working_proxies.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        proxylist.append(row[0])
print(f'Proxies: {len(proxylist)}')

Proxies: 16


In [9]:
url = 'https://www.nba.com/games?date=2024-11-04'

payload = ""
headers = {
    "cookie": "_tapology_mma_session=%252BnGrxOO8u60FBkwjnPf5U9cMUlW%252B%252F76dZtFNqnrNzBiOvQybvXmEnNM%252Fu1%252BEvOx0w4zOYLO6aIlNCfl8UnsrtSYiMl2eRJHAyiBcnd2iP0A0MCwFxGErsRcK9jbT%252BixWWetj2aX%252FvsQSBYea%252Fe73CRDIdSn95lPxaMgzhrkIGIY2KzurUSeLm0hoWxHQyq01nb7UJfYbF53mL1vhZO1yAYpprixBeuhXy70HLYlQemANkpVvl7tT0Z5DTe68LgVyn8qXKLn1hOvclfkBIfaVwBd1HyV5eIRqOMicbIQ%253D--%252BzoeexZ3ARMoGYGy--kqTFUNtOGR74GqxibycT1g%253D%253D",
    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

}
proxy = random.choice(proxylist)
response = requests.request("GET", url, data=payload, headers=headers, params=querystring, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
print(response)

<Response [200]>


In [15]:
#get game cards
soup = BeautifulSoup(response.content, 'html.parser')
gameCards = soup.find_all('div', class_=re.compile('GamesView_gameCardsContainer__c_9fB'))
for card in gameCards:
    print(card)


<div class="GamesView_gameCardsContainer__c_9fB"><div class="GameCard_gc__UCI46 GameCardsMapper_gamecard__pz1rg" data-is-post="true"><section class="GameCard_gcMain__q1lUW"><a class="GameCard_gcm__SKtfh GameCardMatchup_gameCardMatchup__H0uPe" href="/game/mil-vs-cle-0022400154"><div class="GameCardMatchup_wrapper__uUdW8"><article class="GameCardMatchup_article__Fsvx9"><figure class="MatchupCardTeamLogo_base__WZl01 GameCardMatchup_matchupCardTeamLogo__ZqZXC"><div class="TeamLogo_block__rSWmO"><img alt=" Logo" class="TeamLogo_logo__PclAJ" loading="lazy" src="https://cdn.nba.com/logos/nba/1610612749/primary/L/logo.svg" title=" Logo"/></div></figure><div class="MatchupCardTeamName_base__PBkuX" data-team-id="1610612749"><span class="MatchupCardTeamName_teamName__9YaBA">Bucks</span></div><p class="MatchupCardTeamRecord_record__20YHe">-</p></article><div class="GameCardMatchup_statusWrapper__TDbQz"><div class="GameCardMatchup_matchupScoreCardWrapper__X50gw"></div><div class="GameCardMatchup_ga